In [70]:
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
from slsim.Pipelines.skypy_pipeline import SkyPyPipeline
from astropy.units import Quantity
from slsim.lens_pop import LensPop
from slsim.lens import Lens
from slsim.Plots.lens_plots import LensingPlots
import numpy as np
from slsim.selection import deflector_cut
import corner
from lenstronomy.Util import constants
from colossus.cosmology import cosmology

# Galaxy-galaxy simulations

This notebook walks through the basics of simulating a galaxy-galaxy strong lensing population.
The underlying 

assumptions of the galaxy populations (for both lenses and sources) are drawn from a population pre-configured

 and rendered through [SkyPy](https://github.com/skypyproject/skypy). The specific settings are described in the [readme file](https://github.com/LSST-strong-lensing/slsim/tree/gg-lensing/data/SkyPy).

The notebook goes in three steps:

1. The populations of lenses and sources is produced.
2. Random draws of the population are generated and realized as images
3. The full population is generated in catalogue form
4. the full population is represented in a corner plot



## Generate population of galaxies and (potential) deflectors
The LensPop() class in the slsim package is used to produce a set of galaxies (as lenses and sources)

as seen on the sky within a certain sky area.
We use the default SkyPy configuration file. Alternative configuration

files can be used.

In [72]:
# define a cosmology

cosmo_col = cosmology.setCosmology('planck18')
cosmo = FlatLambdaCDM(H0=cosmo_col.H0, Om0=cosmo_col.Om0)

# define a sky area
sky_area = Quantity(value=0.1, unit="deg2")


# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = {"band": "g", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "g", "band_max": 28, "z_min": 0.1, "z_max": 5.0}

skypy_config='/Users/ktabe/Research/strong-lens-SNe/slsim/data/SkyPy/lsst-like.yml'
# run skypy pipeline and make galaxy-galaxy population class using LensPop
gg_lens_pop = LensPop(
    deflector_type="all-galaxies",
    source_type="galaxies",
    kwargs_deflector_cut=kwargs_deflector_cut,
    kwargs_source_cut=kwargs_source_cut,
    kwargs_mass2light=None,
    skypy_config=skypy_config,
    sky_area=sky_area,
    cosmo=cosmo,
)

In [73]:
filters=None
pipeline = SkyPyPipeline(
                skypy_config=skypy_config,
                sky_area=sky_area,
                filters=filters,
                cosmo=cosmo,
            )

In [48]:
gal_list = pipeline.red_galaxies
gal_list.sort("z")
gal_list.reverse()
gal_list



z,M,coeff,ellipticity,physical_size,stellar_mass,angular_size,mag_g,mag_r,mag_i,mag_z,mag_Y
,,,,kpc,,rad,,,,,
float64,float64,float64[5],float64,float64,float64,float64,float64,float64,float64,float64,float64
4.999800459443261,-25.239712765784798,0.03020282316223047 .. 0.0007407315916694893,0.07497594358595468,23.29359197480283,8461609194030.7705,1.7974552071088878e-05,27.307631018660075,24.92249263667381,24.64428758116278,24.631137841343694,24.674646364969348
4.983312396704012,-23.57581358630234,0.04749648124938698 .. 0.0006222250944322844,0.10916459397763396,13.491485234076519,1847915400264.1614,1.0393812817222237e-05,29.4551022555523,27.21258387950662,26.943729328447244,26.922386496920158,26.92700603598875
4.98260526024358,-23.29123893022552,0.0783611261257195 .. 0.0007291037319569196,0.07361934872874885,11.063313875652996,1340918092791.3696,8.522559593244222e-06,27.98393970794679,25.488102413576875,25.21783421938548,25.235598121551106,25.279223583683887
4.975226518952471,-21.212654190687623,0.05215973023800485 .. 0.0004656238630493206,0.2433196699213895,2.492626395545829,201483322790.28452,1.918781762458522e-06,30.91013462777808,28.494101466476852,28.22859212475895,28.242356433385975,28.2627347960987
4.973463934801499,-22.799942156008896,0.006431096283452608 .. 0.0001302275536158153,0.37010347056794,7.24183364743577,934134465139.4835,5.573671006790078e-06,30.90818703264483,29.118574097142496,28.895875911332215,28.889828927783345,28.89130610723591
4.972476329501685,-23.479487751688616,0.06202038183335238 .. 0.0004839198640334943,0.2860161111534387,11.082465740043299,1745827566673.9336,8.528778119138993e-06,29.221797896505052,26.914253042893222,26.652133725301802,26.66181939228172,26.68645401263221
4.961421132747331,-20.415537742988903,0.09461025095002537 .. 0.0002590149407255108,0.13450187610303066,2.743606809033856,105027114540.67049,2.109103312642805e-06,31.879118253765174,29.516504461661935,29.259439605420962,29.289768605060782,29.311061795062923
4.9608778130370625,-20.34689881204559,0.024663092489725102 .. 0.00018430466544998734,0.1422354063333618,2.4062506520067566,95993746548.06244,1.849666883688738e-06,32.206747809996415,29.8968657555623,29.64314991381946,29.672265042129535,29.68898016429255


In [75]:
import pandas as pd
# columns = [
#     'nim', 'mori', 'mobs', 'sep', 'ein', 'fs', 'frac_sh_trunc', 'mmin', 'flag_out', 'zs', 'xs', 'ys'
# ]

# for i in range(5):
#     suffix = "_sh" if i == 0 else "_sat" if i == 1 else "_hh" if i == 2 else "_cen" if i == 3 else "_pert"
#     columns.extend([
#         f'm_type{suffix}', f'zl{suffix}', f'mass{suffix}', f'xl{suffix}',
#         f'yl{suffix}', f'elip{suffix}', f'pol{suffix}', f'param1{suffix}'
#     ])
# file_path = '/Users/ktabe/Research/strong-lens-SNe/genmock_halo/result/default_kapst/'
# prefix = 'default_LSST_kapst'
# data = np.loadtxt(file_path+"/"+prefix+"_result.dat")
# df = pd.DataFrame(data, columns=columns)
# lens_class = df
# lens_class
df = pd.read_csv("/Users/ktabe/Research/strong-lens-SNe/slsim_KTA/data/SL-Hammocks/qso_salpeter_LSST_multi.csv")

In [76]:
mlens_halo = np.where(df["mass_sh"]==-1, df["mass_hh"], df["mass_sh"])
mlens_gal = np.where(df["mass_sh"]==-1, df["mass_cen"], df["mass_sat"])
lens_tb = np.where(df["mass_sh"]==-1, df["param1_cen"], df["param1_sat"])
lens_con = np.where(df["mass_sh"]==-1, df["param1_hh"], df["param1_sh"])
df["mlens_halo"]=mlens_halo
df["mlens_gal"]=mlens_gal
df["lens_tb"]=lens_tb
df["lens_con"]=lens_con
df = df.rename(columns={'zl_hh': 'z'})

In [77]:
halo_galaxy_list = df

In [74]:

z_te = 0.02
constants.arcsec,cosmo.angular_diameter_distance(z_te).value,cosmo_col.angularDiameterDistance(z_te)/(cosmo_col.H0/100.)

(4.84813681109536e-06, 86.47274031460155, 86.47912716830542)

In [59]:
n = len(halo_galaxy_list)
column_names = halo_galaxy_list.columns
if "vel_disp" not in column_names:
    halo_galaxy_list["vel_disp"] = -np.ones(n)
else:
    print("include")

include


In [49]:
kwargs_deflector_cut = {"z_min": 0.01, "z_max": 2.5}
galaxy_select = deflector_cut(halo_galaxy_list, **kwargs_deflector_cut)
galaxy_select.columns

Index(['Unnamed: 0', 'nim', 'mori', 'mobs', 'sep', 'ein', 'fs',
       'frac_sh_trunc', 'mmin', 'flag_out', 'zs', 'xs', 'ys', 'm_type_sh',
       'zl_sh', 'mass_sh', 'xl_sh', 'yl_sh', 'elip_sh', 'pol_sh', 'param1_sh',
       'm_type_sat', 'zl_sat', 'mass_sat', 'xl_sat', 'yl_sat', 'elip_sat',
       'pol_sat', 'param1_sat', 'm_type_hh', 'z', 'mass_hh', 'xl_hh', 'yl_hh',
       'elip_hh', 'pol_hh', 'param1_hh', 'm_type_cen', 'zl_cen', 'mass_cen',
       'xl_cen', 'yl_cen', 'elip_cen', 'pol_cen', 'param1_cen', 'm_type_pert',
       'zl_pert', 'mass_pert', 'xl_pert', 'yl_pert', 'elip_pert', 'pol_pert',
       'param1_pert', 'vel_disp'],
      dtype='object')

lens_class.(by='', ascending=True, inplace=True)

In [ ]:
# define a cosmology
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

# define a sky area
sky_area = Quantity(value=0.1, unit="deg2")


# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = {"band": "g", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "g", "band_max": 28, "z_min": 0.1, "z_max": 5.0}

# run skypy pipeline and make galaxy-galaxy population class using LensPop
gg_lens_pop = LensPop(
    deflector_type="halomodel",
    source_type="galaxies",
    kwargs_deflector_cut=kwargs_deflector_cut,
    kwargs_source_cut=kwargs_source_cut,
    kwargs_mass2light=None,
    skypy_config='/Users/ktabe/Research/strong-lens-SNe/slsim/data/SkyPy/lsst-like.yml',
    sky_area=sky_area,
    cosmo=cosmo,
)

In [4]:
# define a cosmology
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

# define a sky area
sky_area = Quantity(value=0.1, unit="deg2")


# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = {"band": "g", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "g", "band_max": 28, "z_min": 0.1, "z_max": 5.0}

# run skypy pipeline and make galaxy-galaxy population class using LensPop
gg_lens = Lens(
    source_dict,
    deflector_dict,
    cosmo,
    deflector_type="EPL",
    source_type="extended",
    lens_equation_solver="lenstronomy_analytical",
    variability_model=None,
    kwargs_variability=None,
    sn_type=None,
    sn_absolute_mag_band=None,
    sn_absolute_zpsys=None,
    test_area=4 * np.pi,
    mixgauss_means=None,
    mixgauss_stds=None,
    mixgauss_weights=None,
    magnification_limit=0.01,
    light_profile="single_sersic",
    lightcurve_time=None,
    deflector_type="all-galaxies",
    source_type="galaxies",
    kwargs_deflector_cut=kwargs_deflector_cut,
    kwargs_source_cut=kwargs_source_cut,
    kwargs_mass2light=None,
    skypy_config='/Users/ktabe/Research/strong-lens-SNe/slsim/data/SkyPy/lsst-like.yml',
    sky_area=sky_area,
    cosmo=cosmo,
)

TypeError: Lens.__init__() got an unexpected keyword argument 'deflector_type'

## Generate images of random lenses
The LensingPlots() class has the functionality to draw random lenses and makes an image of it.
Currently

default settings in lenstronomy are chosen for the LSST image settings. These will be able to be replaced with the

LSST simulation tools.

In [ ]:
# make some cuts in the image separations and limited magnitudes of the arc
kwargs_lens_cut_plot = {
    "min_image_separation": 0.8,
    "max_image_separation": 10,
    "mag_arc_limit": {"g": 23, "r": 23, "i": 23},
}


gg_plot = LensingPlots(gg_lens_pop, num_pix=64, coadd_years=10)

# generate montage indicating which bands are used for the rgb color image
fig, axes = gg_plot.plot_montage(
    rgb_band_list=["i", "r", "g"],
    add_noise=True,
    n_horizont=5,
    n_vertical=2,
    kwargs_lens_cut=kwargs_lens_cut_plot,
)
plt.show()

## Generate the full population
We are using the instance of the LensPop() class to draw the full population within specified cuts in a Monte Carlo process.

In [ ]:
# specifying cuts of the population
kwargs_lens_cuts = {"mag_arc_limit": {"g": 28}}
# drawing population
gg_lens_population = gg_lens_pop.draw_population(kwargs_lens_cuts=kwargs_lens_cuts)

## Represent key quantities of full population in corner plots
We calculate few key quantities of the lenses. The full population is represented each with a Lens() class

object that allows to compute and return these (and more) quantities.

In [ ]:
print("Number of lenses:", len(gg_lens_population))

lens_samples = []
labels = [
    r"$\sigma_v$",
    r"$\log(M_{*})$",
    r"$\theta_E$",
    r"$z_{\rm l}$",
    r"$z_{\rm s}$",
    r"$m_{\rm source}$",
    r"$m_{\rm lens}$",
]

for gg_lens in gg_lens_population:
    vel_disp = gg_lens.deflector_velocity_dispersion()
    m_star = gg_lens.deflector_stellar_mass()
    theta_e = gg_lens.einstein_radius
    zl = gg_lens.deflector_redshift
    zs = gg_lens.source_redshift
    source_mag = gg_lens.extended_source_magnitude(band="g", lensed=True)
    deflector_mag = gg_lens.deflector_magnitude(band="g")
    lens_samples.append(
        [vel_disp, np.log10(m_star), theta_e, zl, zs, source_mag, deflector_mag]
    )

In [ ]:
hist2dkwargs = {
    "plot_density": False,
    "plot_contours": False,
    "plot_datapoints": True,
    "color": "b",
    "data_kwargs": {"ms": 5},
}
corner.corner(np.array(lens_samples), labels=labels, **hist2dkwargs)
plt.show()